<a href="https://colab.research.google.com/github/ykitaguchi77/chatGPT/blob/main/OPENAI_agent_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**OPENAI_agent_sdk**

https://github.com/openai/openai-agents-python?tab=readme-ov-file

In [ ]:
import os
from google.colab import userdata

# OpenAI APIキーをColabのシークレットから取得して設定
api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key

print("APIキーが設定されました")

def set_api():
  os.environ["OPENAI_API_KEY"] = api_key


APIキーが設定されました


In [ ]:
!pip install openai-agents --q

In [ ]:
from agents import Agent, Runner

set_api()

agent = Agent(name="Assistant", instructions="あなたは面白い漫才師です")

# Intended for Jupyter notebooks where there's an existing event loop
result = await Runner.run(agent, "面白いダジャレを言って下さい.") # type: ignore[top-level-await]  # noqa: F704
print(result.final_output)


もちろん！ではこちらをどうぞ：

バナナが滑ったから、バナナスッテンコロリン！どうかな？


In [ ]:
# 必要なライブラリのインポート
from agents import Agent, Runner
import asyncio
import nest_asyncio

# Jupyter内で非同期コードを実行するために必要
nest_asyncio.apply()

# API設定
set_api()

# エージェントの定義
spanish_agent = Agent(
    name="Spanish agent",
    instructions="You only speak Spanish.",
)

english_agent = Agent(
    name="English agent",
    instructions="You only speak English",
)

triage_agent = Agent(
    name="Triage agent",
    instructions="Handoff to the appropriate agent based on the language of the request.",
    handoffs=[spanish_agent, english_agent],
)

# 方法1: 直接awaitを使用してJupyterセル内で実行（nest_asyncioを使用）
result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
print(result.final_output)
# 期待される出力: ¡Hola! Estoy bien, gracias por preguntar. ¿Y tú, cómo estás?

# 方法2: 関数を定義して再利用可能にする
async def run_with_language(text):
    result = await Runner.run(triage_agent, input=text)
    return result.final_output

# スペイン語の例
spanish_response = await run_with_language("Hola, ¿cómo estás?")
print(f"スペイン語の応答: {spanish_response}")

# 英語の例
english_response = await run_with_language("Hello, how are you?")
print(f"英語の応答: {english_response}")

# 方法3: 元のコードの形式を保持（別のセルで実行する場合）
async def main():
    result = await Runner.run(triage_agent, input="Hola, ¿cómo estás?")
    print(result.final_output)
    return result

# この行をコメント解除して実行すると、伝統的な方法で実行できます
# asyncio.run(main())

¡Hola! Estoy bien, gracias. ¿Y tú?
スペイン語の応答: ¡Hola! Estoy bien, gracias. ¿Y tú?
英語の応答: Hello! I'm here to help. How are you doing today?


In [ ]:
import asyncio

from agents import Agent, Runner, function_tool

# Jupyter内で非同期コードを実行するために必要
nest_asyncio.apply()

set_api()

@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny."


agent = Agent(
    name="Hello world",
    instructions="You are a helpful agent.",
    tools=[get_weather],
)


async def main():
    result = await Runner.run(agent, input="What's the weather in Tokyo?")
    print(result.final_output)
    # The weather in Tokyo is sunny.


if __name__ == "__main__":
    asyncio.run(main())

The weather in Tokyo is sunny. ☀️


In [ ]:
# OpenAI Agentsのインストール
!pip install openai-agents nest-asyncio --q

# 必要なライブラリのインポート
import os
import asyncio
import nest_asyncio
from google.colab import userdata
from agents import Agent, Runner, function_tool

# Jupyter内での非同期コード実行のための設定
nest_asyncio.apply()

# OpenAI APIキーの設定
api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = api_key
print("APIキーが設定されました")

# ツール関数の定義
@function_tool
def translate_to_japanese(text: str) -> str:
    """テキストを日本語に翻訳します。"""
    return f"「{text}」の日本語訳：これは翻訳された文章です。みつを"

@function_tool
def get_current_weather(city: str) -> str:
    """指定された都市の現在の天気を返します。"""
    weather_data = {
        "tokyo": "東京は晴れ、気温は44度です。",
        "osaka": "大阪は曇り、気温は23度です。",
        "kyoto": "京都は雨、気温は20度です。",
        "new york": "ニューヨークは晴れ、気温は22度です。",
        "london": "ロンドンは曇り、気温は15度です。",
        "paris": "パリは晴れ、気温は18度です。"
    }

    city_lower = city.lower()
    if city_lower in weather_data:
        return weather_data[city_lower]
    else:
        return f"{city}の天気情報は利用できません。"

@function_tool
def calculate_sum(numbers: str) -> str:
    """カンマ区切りの数字の合計を計算します。"""
    try:
        num_list = [float(n.strip()) for n in numbers.split(',')]
        total = sum(num_list)
        return f"合計: {total}"
    except Exception as e:
        return f"計算エラー: {str(e)}"

# 日本語エージェント
japanese_agent = Agent(
    name="Japanese Agent",
    instructions="""
    あなたは日本語のエキスパートエージェントです。
    日本語での会話やサポートを提供します。
    質問に対して日本語で回答してください。
    必要に応じて天気予報ツールを使用できます。
    """,
    tools=[get_current_weather]
)

# 英語エージェント
english_agent = Agent(
    name="English Agent",
    instructions="""
    You are an English language expert agent.
    Provide conversations and support in English.
    Answer questions in English.
    You can use the weather tool when needed.
    """,
    tools=[get_current_weather]
)

# 計算エージェント
math_agent = Agent(
    name="Math Agent",
    instructions="""
    あなたは計算の専門家エージェントです。
    数学的な質問や計算の依頼に対応します。
    計算ツールを使って正確な結果を提供してください。
    """,
    tools=[calculate_sum]
)

# 翻訳エージェント
translator_agent = Agent(
    name="Translator Agent",
    instructions="""
    あなたは翻訳エージェントです。
    様々な言語間の翻訳を行います。
    特に英語からイタリア語への翻訳を専門としています。
    翻訳ツールを使用して正確な翻訳を提供してください。
    """,
    tools=[translate_to_japanese]
)

# トリアージエージェント
triage_agent = Agent(
    name="Triage Agent",
    instructions="""
    あなたはリクエストを適切なエージェントに振り分けるトリアージエージェントです。

    以下の基準で振り分けを行ってください：
    - 日本語の質問やリクエスト → Japanese Agent
    - 英語の質問やリクエスト → English Agent
    - 計算や数学に関する質問 → Math Agent
    - 翻訳のリクエスト → Translator Agent

    振り分けの決定は簡潔に行い、適切なエージェントにハンドオフしてください。
    どのエージェントに振り分けたかどうか、およびその理由を解説してください
    """,
    handoffs=[japanese_agent, english_agent, math_agent, translator_agent]
)

# メインアシスタントエージェント
assistant_agent = Agent(
    name="Assistant Agent",
    instructions="""
    あなたは便利なアシスタントエージェントです。
    ユーザーからの様々なリクエストに対応します。

    特定の専門知識が必要な場合は、トリアージエージェントに振り分けてください。
    どのような質問でも必ずエージェントに振り分けて下さい。
    """,
    handoffs=[triage_agent],
    tools=[translate_to_japanese, get_current_weather, calculate_sum]
)

# 単一メッセージでのテスト実行関数
async def test_single_message(message):
    print(f"入力: {message}")
    result = await Runner.run(assistant_agent, input=message)
    print(f"応答: {result.final_output}")
    return result

# 複数のテスト例を実行する関数
async def run_test_examples():
    examples = [
        "こんにちは、今日の東京の天気を教えてください。",
        "Hello, what's the weather in New York?",
        "3, 5, 7, 9 の合計を計算してください。",
        "Please translate this to Japanese: I love visiting Japan and learning about its culture.",
        "五条悟の術式は何ですか？"
    ]

    for i, example in enumerate(examples, 1):
        print(f"\n=== テスト {i} ===")
        await test_single_message(example)

# 対話型セッション
async def interactive_session():
    print("\n=== 対話型アシスタントセッション開始 ===")
    print("終了するには 'exit' または 'quit' と入力してください")

    while True:
        user_input = input("\nあなた: ")
        if user_input.lower() in ['exit', 'quit', '終了']:
            print("セッションを終了します。")
            break

        result = await test_single_message(user_input)

#テスト実行
print("=== マルチエージェントシステムのテスト ===")
await run_test_examples()

# #対話型セッションの実行（必要に応じてコメント解除）
# await interactive_session()

# 個別のメッセージをテスト（必要に応じて使用）
# カスタムメッセージのテスト例
# custom_message = "京都の天気はどうですか？"
# await test_single_message(custom_message)

APIキーが設定されました
=== マルチエージェントシステムのテスト ===

=== テスト 1 ===
入力: こんにちは、今日の東京の天気を教えてください。
応答: こんにちは！今日の東京の天気は晴れで、気温は44度です。暑い日なので、熱中症に気をつけてくださいね。

=== テスト 2 ===
入力: Hello, what's the weather in New York?
応答: The weather in New York is currently sunny with a temperature of 22°C.

=== テスト 3 ===
入力: 3, 5, 7, 9 の合計を計算してください。
応答: 3, 5, 7, 9 の合計は 24 です。

=== テスト 4 ===
入力: Please translate this to Japanese: I love visiting Japan and learning about its culture.
応答: I love visiting Japan and learning about its culture. の日本語訳は次の通りです：
日本を訪れ、その文化について学ぶことが大好きです。

=== テスト 5 ===
入力: 五条悟の術式は何ですか？
応答: 五条悟の術式は、「無下限呪術」および「六眼」です。彼の能力は、その強力な術式と視覚によって非常に詳細な情報を理解し、操作することができます。具体的には「無為転変」や「蒼」、「赫」といった技を駆使することができます。
